In [2]:
from nltk.text import TextCollection
from nltk.tokenize import word_tokenize
 
#首先，构建语料库corpus
sents=['this is sentence one','this is sentence two','this is sentence three']
sents=[word_tokenize(sent) for sent in sents] #对每个句子进行分词
print(sents)  #输出分词后的结果
corpus=TextCollection(sents)  #构建语料库
print(corpus)  #输出语料库
 
#计算语料库中"one"的tf值
tf=corpus.tf('one',corpus)    # 1/12
print(tf)
 
#计算语料库中"one"的idf值
idf=corpus.idf('one')      #log(3/1)
print(idf)
 
#计算语料库中"one"的tf-idf值
tf_idf=corpus.tf_idf('one',corpus)
print(tf_idf)


[['this', 'is', 'sentence', 'one'], ['this', 'is', 'sentence', 'two'], ['this', 'is', 'sentence', 'three']]
<Text: this is sentence one this is sentence two...>
0.08333333333333333
1.0986122886681098
0.0915510240556758


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)#测试集比例占25%

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
#Evaluate Metrics 评价指标
accuracy.rmse(predictions)# Then compute RMSE
accuracy.mae(predictions)#Then compute NAE

RMSE: 0.9323
MAE:  0.7344


0.7344053841770187

In [1]:
from surprise import KNNBasic
from surprise import Dataset
data = Dataset.load_builtin('ml-100k') #加载movielens-100k数据集
trainset = data.build_full_trainset() #纠正/取出训练集
#sim_options={nam=cosine, user_based=True,min_support=10,shrinkage}
algo = KNNBasic(k=40, min_k=1,sim_options ={'name':'cosine','user_based': True}, verbose=True) #建立UCF算法并训练
algo.fit(trainset)
uid = str(196)  # 原始user id (在评分文件中的). 注意，是个字符串
iid = str(302)  # 原始item id （其他同上）
pred = algo.predict(uid, iid, r_ui=4, verbose=True) #对某一个具体的user和item给出预测

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 4.12   {'actual_k': 40, 'was_impossible': False}


In [2]:
from surprise import KNNBasic
from surprise import Dataset
data = Dataset.load_builtin('ml-100k') #加载movielens-100k数据集
trainset = data.build_full_trainset() #纠正/取出训练集
#sim_options={nam=cosine, user_based=True,min_support=10,shrinkage}
algo = KNNBasic(k=40, min_k=1,sim_options ={'name':'cosine','user_based': False}, verbose=True) #建立ICF算法并训练
algo.fit(trainset)
uid = str(196)  # 原始user id (在评分文件中的). 注意，是个字符串
iid = str(302)  # 原始item id （其他同上）
pred = algo.predict(uid, iid, r_ui=4, verbose=True) #对某一个具体的user和item给出预测

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 3.61   {'actual_k': 39, 'was_impossible': False}


In [10]:
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n
# First train an UCF algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25)#测试集比例占25%
algo = KNNBasic(k=50, min_k=10,sim_options ={'name':'cosine','user_based': True}, verbose=True) #建立UCF算法并训练
algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

#Evaluate Metrics 评价指标
accuracy.rmse(predictions)# Then compute RMSE
accuracy.mae(predictions)#Then compute NAE
n=5 #推荐列表长度
top_n = get_top_n(predictions, n)
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print('UserID:',uid, 'Top-'+str(n)+' Recommender List',[iid for (iid, _) in user_ratings])

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0205
MAE:  0.8083
UserID: 381 Top-5 Recommender List ['483', '318', '480', '14', '493']
UserID: 90 Top-5 Recommender List ['357', '12', '64', '98', '313']
UserID: 533 Top-5 Recommender List ['318', '483', '169', '174', '498']
UserID: 385 Top-5 Recommender List ['654', '483', '169', '318', '197']
UserID: 763 Top-5 Recommender List ['12', '197', '50', '98', '498']
UserID: 703 Top-5 Recommender List ['100', '9', '1', '258', '508']
UserID: 775 Top-5 Recommender List ['315', '258', '344', '887', '348']
UserID: 174 Top-5 Recommender List ['12', '272', '98', '196', '269']
UserID: 186 Top-5 Recommender List ['12', '79', '95', '257', '588']
UserID: 130 Top-5 Recommender List ['134', '22', '316', '195', '185']
UserID: 200 Top-5 Recommender List ['318', '98', '515', '496', '45']
UserID: 416 Top-5 Recommender List ['178', '64', '174', '479', '183']
UserID: 263 Top-5 Recommender List ['357', '127', '97', '79', '210'

UserID: 88 Top-5 Recommender List ['300', '326', '886', '308']
UserID: 695 Top-5 Recommender List ['242', '305', '311', '286', '270']
UserID: 909 Top-5 Recommender List ['529', '292', '294', '682', '289']
UserID: 32 Top-5 Recommender List ['313', '257', '307', '742', '118']
UserID: 926 Top-5 Recommender List ['315', '300', '340', '262', '245']
UserID: 603 Top-5 Recommender List ['50', '474', '172', '174', '100']
UserID: 420 Top-5 Recommender List ['513', '285', '190', '19', '475']
UserID: 726 Top-5 Recommender List ['1038', '255', '898', '323', '832']
UserID: 61 Top-5 Recommender List ['347', '300', '331', '328', '271']
UserID: 419 Top-5 Recommender List ['50', '69', '306', '494', '615']
UserID: 341 Top-5 Recommender List ['292', '259', '948']
UserID: 441 Top-5 Recommender List ['15', '117', '282', '25', '538']
UserID: 350 Top-5 Recommender List ['187', '654', '50', '179', '168']
UserID: 418 Top-5 Recommender List ['302', '269', '1313', '301', '327']
UserID: 208 Top-5 Recommender List 